In [1]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pandas as pd
from sklearn import preprocessing

from xgboost import plot_importance
import numpy as np
from sklearn.metrics import confusion_matrix


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm, tqdm_notebook, trange
from scipy.optimize import fmin
from pylab import rcParams
from IPython.display import clear_output
import copy
rcParams['figure.figsize'] = 8, 5

C:\Users\shankar\Anaconda3\envs\tf\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_excel('sample_submission.xlsx')
train.shape, test.shape, sample.shape

((33908, 17), (11303, 16), (11303, 1))

In [5]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns)+['_'.join(x) for x in combos]
    
    #scaler = MinMaxScaler()
    #scaler.fit(df)
    #df = scaler.transform(df)
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indices = [i for i,x in enumerate(list((df==0).all())) if x]
    df= df.drop(df.columns[noint_indices], axis=1)
    
    return df

In [7]:
X = train.drop('labels', axis=1)
y = train['labels']

In [8]:
data = pd.concat([X, test]).reset_index(drop=True)
data.shape

(45211, 16)

In [9]:
data = add_interactions(data)
data.shape

(45211, 136)

In [10]:
X = data[:train.shape[0]]
test_X = data[train.shape[0]:].reset_index(drop=True)

In [11]:
X['labels'] = y
X.shape

C:\Users\shankar\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(33908, 137)

In [14]:
X.to_csv('train_136_feat.csv', index=False)
test_X.to_csv('test_136_feat.csv', index=False)

In [15]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_excel('sample_submission.xlsx')
train.shape, test.shape, sample.shape

((33908, 17), (11303, 16), (11303, 1))

In [16]:
num_cols1 = ['feature_1', 'feature_3', 'feature_5']
num_cols2 = ['feature_0', 'feature_4', 'feature_6']

cat_cols1 = ['feature_2', 'feature_7', 'feature_14']
cat_cols2 = ['feature_8', 'feature_13']
cat_cols3 = ['feature_9', 'feature_15']
cat_cols4 = ['feature_10', 'feature_11', 'feature_12']

In [17]:
def feature_eng_1(df):
    df['sum_num_1'] = df[num_cols1].sum(axis=1)
    df['sum_num_2'] = df[num_cols2].sum(axis=1)
    df['sum_num_3'] = df[num_cols1+num_cols2].sum(axis=1)
    
    df['sum_cat1'] = df[cat_cols2+cat_cols3].sum(axis=1)
    df['sum_cat2'] = df[cat_cols4].sum(axis=1)
    df['sum_cat3'] = df[cat_cols2+cat_cols3+cat_cols4].sum(axis=1)
    df['sum_cat4'] = df[cat_cols2+cat_cols3+cat_cols4].sum(axis=1)
    
    df['mean_num_1'] = df[num_cols1].mean(axis=1)
    df['mean_num_2'] = df[num_cols2].mean(axis=1)
    df['mean_num_3'] = df[num_cols1+num_cols2].mean(axis=1)
    
    df['std_num_1'] = df[num_cols1].std(axis=1)
    df['std_num_2'] = df[num_cols2].std(axis=1)
    df['std_num_3'] = df[num_cols1+num_cols2].std(axis=1)
    
    return df

In [18]:
train = feature_eng_1(train)
test = feature_eng_1(test)

train.shape, test.shape

((33908, 30), (11303, 29))

In [26]:
train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,labels,sum_num_1,sum_num_2,sum_num_3,sum_cat1,sum_cat2,sum_cat3,sum_cat4,mean_num_1,mean_num_2,mean_num_3,std_num_1,std_num_2,std_num_3
0,-0.276515,-0.424429,1.344997,-0.012283,0.076230,1.076648,0.182198,3,0,1,0,0,0,0,10,2,1,0.639936,-0.018087,0.621849,3,0,3,3,0.213312,-0.006029,0.103641,0.775550,0.240165,0.527348
1,0.853573,0.150991,0.503892,-0.979179,-0.569351,-0.411453,-0.251940,4,1,2,0,1,0,0,0,3,0,-1.239641,0.032282,-1.207359,6,1,7,7,-0.413214,0.010761,-0.201226,0.565087,0.746952,0.636263
2,0.947747,-0.173832,1.825628,-0.703478,0.076230,-0.411453,-0.251940,6,1,2,0,0,0,0,5,3,0,-1.288763,0.772037,-0.516726,6,0,6,6,-0.429588,0.257346,-0.086121,0.265288,0.620012,0.568753
3,0.853573,-0.381404,0.984523,-0.039464,-0.569351,-0.411453,-0.251940,4,0,2,0,1,0,0,5,3,0,-0.832321,0.032282,-0.800039,5,1,6,6,-0.277440,0.010761,-0.133340,0.206640,0.746952,0.514949
4,1.324443,1.590527,-1.178318,-0.097711,-0.246560,-0.411453,-0.251940,0,1,1,0,0,0,0,8,3,0,1.081362,0.825942,1.907305,5,0,5,5,0.360454,0.275314,0.317884,1.076762,0.908576,0.892270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33903,1.701139,-0.248387,-0.457371,-0.792789,-0.569351,4.851829,0.616335,5,0,0,0,0,0,2,10,0,0,3.810652,1.748124,5.558776,2,0,2,2,1.270217,0.582708,0.926463,3.113687,1.135618,2.129712
33904,1.512791,-0.433625,-1.298476,-0.823854,-0.569351,-0.411453,-0.251940,10,1,2,0,0,0,2,8,3,0,-1.668933,0.691500,-0.977433,8,0,8,8,-0.556311,0.230500,-0.162905,0.231965,1.121780,0.842971
33905,0.006007,-0.395526,0.984523,-0.555919,-0.246560,-0.411453,-0.251940,0,1,1,0,1,0,0,7,3,0,-1.362899,-0.492493,-1.855392,5,1,6,6,-0.454300,-0.164164,-0.309232,0.088365,0.147398,0.192529
33906,-0.088167,0.789143,0.503892,-0.664647,-0.246560,-0.411453,-0.251940,1,1,1,0,1,0,0,9,3,0,-0.286957,-0.586667,-0.873624,5,1,6,6,-0.095652,-0.195556,-0.145604,0.776642,0.093041,0.497721


In [41]:
cols = ['feature_0', 'feature_1', 'feature_3', 'feature_5']
for i in cols:
    for j in cols:
        if i != j:
            train[i+'_div_'+j] = train[i]/train[j]
            test[i+'_div_'+j] = test[i]/test[j]

In [42]:
train.shape, test.shape

((33908, 42), (11303, 41))

In [43]:
cols = ['feature_2', 'feature_4', 'feature_6']
for i in cols:
    for j in cols:
        if i != j:
            train[i+'_div_'+j] = train[i]/train[j]
            test[i+'_div_'+j] = test[i]/test[j]

In [44]:
train.shape, test.shape

((33908, 48), (11303, 47))

In [45]:
train.to_csv('train_47_feat.csv', index=False)
test.to_csv('test_47_feat.csv', index=False)